In [49]:
!pip install requests
!pip install lxml 
!pip install beautifulsoup4
!pip install google-search-results
!pip install PyDrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#IMPORTAR LAS LIBRERIAS
import requests, lxml
from bs4 import BeautifulSoup
from serpapi import GoogleSearch # API soulution
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
#LIBRERIAS PARA PROCESAMIENTO DE DATOS
import pandas as pd
#LIBRERIAS DE TIEMPO DE PROCESAMIENTO
import time
from tqdm import tqdm, trange, tqdm_notebook
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import sys
#LIBRERIAS PARA WORD2 VEC
from collections import Counter
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import json

In [51]:
#CREAR EL CERTIFICADO DE AUTENTICACIÓN
#auth.authenticate_user()
from google.auth import credentials
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [52]:
#FUNCIÓN DE SCRAPING DE INDICADORES EN GOOGLE SCHOLAR
def scraper(title):
  params = {
  "engine": "google_scholar",
  "q": title,
  "api_key": "2422b653aeaae870eed714743a32d0b023507f38df973c63ef8282e2b177fd0b"}
  search = GoogleSearch(params)
  results = search.get_dict()
  return results

In [53]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1ZtP3VBmO6XSklt8_Zt-S1glwwIJe3wlW"})   # replace the id with id of file you want to access
downloaded.GetContentFile('data_completa_google3.csv')

In [1]:
from io import open_code
import csv
import pandas as pd
db1=pd.read_csv('data_completa_google_8_.csv', sep=',',index_col=False) #('data_limpia2.csv')

In [111]:
#EJECUTAR LA FUNCION EN LOS TITULOS DE LA TESIS Y GUARDAR EN UNA COLUMNA
#db1['organic_results'][14294:len(db1)]=db1['dc.title'][14294:len(db1)].progress_apply(lambda x:scraper(x))
#db1['organic_results'][15298:16300]=db1['dc.title'][15298:16300].progress_apply(lambda x:scraper(x))
#db1['organic_results'][16300:17304]=db1['dc.title'][16300:17304].progress_apply(lambda x:scraper(x))
#db1['organic_results'][17305:18315]=db1['dc.title'][17305:18315].progress_apply(lambda x:scraper(x))
db1['organic_results'][18640:len(db1)]=db1['dc.title'][18640:len(db1)].progress_apply(lambda x:scraper(x))

  0%|          | 0/18 [00:00<?, ?it/s]

<ipython-input-111-845d3c73fcc5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db1['organic_results'][18640:len(db1)]=db1['dc.title'][18640:len(db1)].progress_apply(lambda x:scraper(x))


In [119]:
db1.to_excel('data_completa_google_8_.xlsx')

In [8]:
import ast
def parseo(registro):
  json_data = json.loads(json.dumps(registro))
  json_dat2=json_data.replace("'",'"')
  #json_dat3=json_dat2.replace('" …"','"')
  #orig_new = json_dat3.replace(".","_")
  return json_dat2

In [9]:
#db['organic_results_json']
db['organic_results_json']=db['organic_results'].progress_apply(lambda x:parseo(x))

  0%|          | 0/18658 [00:00<?, ?it/s]

In [16]:
db = db.iloc[1: , :]

In [17]:
def get_versiones(registro):
  try:
    customer = json.loads(registro)
    try:
      if 'total' in customer['organic_results'][0]['inline_links']['versions']:
        element=customer['organic_results'][0]['inline_links']['versions']['total']
    except KeyError:
      element=0
  except ValueError:
    element=0
  return element

In [18]:
def get_citas(registro):
  try:
    #reg=r''
    customer = json.loads(registro)
    try:
      if 'total' in customer['organic_results'][0]['inline_links']['cited_by']:
        element=customer['organic_results'][0]['inline_links']['cited_by']['total']
    except KeyError:
      element=0
  except ValueError:
    element=0
  return element

In [19]:
db['Citas']=db['organic_results_json'].progress_apply(lambda x:get_citas(x))
db['Versiones']=db['organic_results_json'].progress_apply(lambda x:get_versiones(x))

  0%|          | 0/18657 [00:00<?, ?it/s]

<ipython-input-19-85590d568f6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['Citas']=db['organic_results_json'].progress_apply(lambda x:get_citas(x))


  0%|          | 0/18657 [00:00<?, ?it/s]

<ipython-input-19-85590d568f6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['Versiones']=db['organic_results_json'].progress_apply(lambda x:get_versiones(x))


In [22]:
db.to_excel("data_completa_google_final_todo.xlsx")

In [21]:
db['Citas'].describe()

count    18657.000000
mean         1.972343
std         22.354544
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max       1492.000000
Name: Citas, dtype: float64